# Descargar Modelo Digital de Elevación 

In [1]:
# Importar GEE y Iniciar sesion
import ee, os, glob, geemap
import geopandas as gpd

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map(basemap='Esri.WorldImagery')

## Importar shapefile

In [4]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Script'

In [8]:
ruta_archivos = r"C:\Users\GEOMATICA\Curso_Geomatica\06_BGR_Geomatica\Datos\Shp\Zona_buffer_500"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Shp\\Zona_buffer_500'

In [9]:
lista = glob.glob("*.shp")
lista

['Zona_buffer_500.shp']

In [10]:
# cargar un shp
zona_gdp = gpd.read_file(lista[0])
zona_gdp

,FID,geometry
0,0,"POLYGON ((203320.135 9045543.001, 203320.135 9..."


## Convertir en Geográfico WGS84

In [11]:
zona_gdp.crs.name

'WGS 84 / UTM zone 18S'

In [12]:
# Hagamos una copia de seguridad de nuestros datos
Zona_geo = zona_gdp.copy()

# Reproject the data
Zona_wg84 = Zona_geo.to_crs(epsg=4326)

In [13]:
Zona_wg84.crs.name

'WGS 84'

## Convertir de Geopandas a GEE

In [14]:
# Crear un featureCollecion mediante GeoJSON
ee_zona_gdp = ee.FeatureCollection(Zona_wg84.__geo_interface__)

In [15]:
geometria = ee_zona_gdp.geometry()

In [16]:
type(geometria)

ee.geometry.Geometry

In [17]:
# Visualizar mapa
Map.centerObject(geometria,12)
Map.addLayer(ee_zona_gdp, {"color" : "00FF11"}, "zona_gdp")
Map

Map(center=[-8.60236701028476, -77.66381910743893], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
Limite_zona = geometria.bounds()

In [23]:
Limite_zona2 = Limite_zona.buffer(300)

In [24]:
Map.addLayer(Limite_zona2, {"color" : "blue"}, "Buffer bouns")
Map

Map(bottom=1099144.0, center=[-8.596636953661147, -77.65222549438478], controls=(WidgetControl(options=['posit…

## Obtencion de MDE

In [25]:
SRTM_3 = ee.Image("USGS/SRTMGL1_003")\
            .select('elevation')

In [27]:
# Recortar RS y TB
MDE_recorte = SRTM_3.clip(Limite_zona2)

In [29]:
# Reproyectar segun zona de estudio
MDE_recorte_utm = MDE_recorte.reproject(crs="EPSG:32718", scale=30)

In [30]:
# Consultar Proyeccion
print("Proyeccion MDE:",MDE_Buffer_utm .projection().getInfo())

Proyeccion MDE: {'type': 'Projection', 'crs': 'EPSG:32718', 'transform': [30, 0, 0, 0, -30, 0]}


In [31]:
# Visualizacion TB y RS
Map.addLayer(MDE_Buffer_utm, {},"MDE Utm")
Map

Map(bottom=1099144.0, center=[-8.596636953661147, -77.65222549438478], controls=(WidgetControl(options=['posit…

## Exportar Imagen calibrado

In [56]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Curso_Geomatica\\06_BGR_Geomatica\\Datos\\Shp'

In [32]:
ruta_archivos = r"D:\Descarga"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'D:\\Descarga'

In [35]:
# Exportar imagen
geemap.ee_export_image(MDE_Buffer_utm, 
                       filename="MDE_SRTM.tif", 
                       scale=30, 
                       region=geometria, 
                       file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to D:\Descarga\MDE_SRTM1.tif
